<div align="center" style="color: pink">

# **Zadanie nr 1**

</div>

In [ ]:
import torch
from torch.nn import functional as F
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, set_seed
import random

MODEL_NAME = "flax-community/papuGaPT2"
DEVICE = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")
SEED = 42

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

In [228]:
# KONFIGURACJE

EXAMPLE_NUM = 20
TEST_NUM = 20

In [229]:
import re

def parse_number(s):
    match = re.search(r"-?\d+", s)
    return int(match.group()) if match else None

In [230]:
def example_generator(min_val: int = 0, max_val: int = 100, operator: str = "+") -> str:
    num1 = random.randint(min_val, max_val)
    num2 = random.randint(min_val, max_val)

    match operator:
        case "+":
            ans = num1 + num2
        case "-":
            ans = num1 - num2
        case "*":
            ans = num1 * num2
        case "/":
            ans = num1 / num2
        case _:
            raise 
        
    return f"{num1} {operator} {num2} = {ans}\n"

EXAMPLE_FEWSHOTS = set()
for _ in range(EXAMPLE_NUM):
    EXAMPLE_FEWSHOTS.add(example_generator())

EXAMPLE_FEWSHOTS = list(EXAMPLE_FEWSHOTS)

print(EXAMPLE_FEWSHOTS)
EXAMPLE_FEWSHOTS = ['10 + 8 = 18\n', '0 + 6 = 6\n', '1 + 8 = 9\n', '8 + 9 = 17\n', '6 + 9 = 15\n', '2 + 1 = 3\n', '3 + 1 = 4\n', '7 + 0 = 7\n', '2 + 2 = 4\n', '3 + 2 = 5\n', '0 + 3 = 3\n', '9 + 9 = 18\n', '8 + 7 = 15\n', '2 + 4 = 6\n', '10 + 7 = 17\n', '4 + 2 = 6\n', '0 + 2 = 2\n', '9 + 2 = 11\n', '3 + 9 = 12\n', '9 + 0 = 9\n', '1 + 10 = 11\n', '0 + 1 = 1\n', '5 + 4 = 9\n', '6 + 10 = 16\n', '8 + 1 = 9\n', '1 + 0 = 1\n', '2 + 9 = 11\n', '5 + 8 = 13\n', '1 + 9 = 10\n', '3 + 10 = 13\n', '2 + 3 = 5\n', '8 + 8 = 16\n', '1 + 1 = 2\n', '7 + 10 = 17\n', '3 + 6 = 9\n', '5 + 9 = 14\n', '1 + 2 = 3\n', '0 + 5 = 5\n', '0 + 8 = 8\n', '8 + 0 = 8\n', '10 + 3 = 13\n', '0 + 0 = 0\n', '7 + 5 = 12\n', '8 + 3 = 11\n', '6 + 6 = 12\n', '5 + 6 = 11\n', '7 + 4 = 11\n', '10 + 10 = 20\n', '9 + 3 = 12\n', '0 + 10 = 10\n', '4 + 4 = 8\n', '10 + 9 = 19\n', '6 + 0 = 6\n', '5 + 0 = 5\n', '2 + 0 = 2\n', '2 + 5 = 7\n', '3 + 8 = 11\n', '9 + 7 = 16\n', '2 + 7 = 9\n', '6 + 4 = 10\n', '9 + 5 = 14\n', '8 + 2 = 10\n', '3 + 5 = 8\n', '10 + 5 = 15\n', '5 + 7 = 12\n', '8 + 6 = 14\n', '4 + 9 = 13\n', '1 + 4 = 5\n', '4 + 7 = 11\n', '5 + 1 = 6\n', '7 + 1 = 8\n']


['50 + 53 = 103\n', '82 + 40 = 122\n', '54 + 2 = 56\n', '46 + 86 = 132\n', '94 + 11 = 105\n', '3 + 6 = 9\n', '88 + 78 = 166\n', '64 + 50 = 114\n', '76 + 24 = 100\n', '24 + 41 = 65\n', '33 + 83 = 116\n', '17 + 13 = 30\n', '42 + 14 = 56\n', '69 + 59 = 128\n', '15 + 31 = 46\n', '95 + 24 = 119\n', '39 + 19 = 58\n', '74 + 81 = 155\n', '4 + 7 = 11\n', '28 + 0 = 28\n']


In [231]:
def exercise_generator(min_val: int = 0, max_val: int = 100, operator: str = "+") -> str:
    num1 = random.randint(min_val, max_val)
    num2 = random.randint(min_val, max_val)

    return f"{num1} {operator} {num2} ="

TEST_TASKS = [exercise_generator() for _ in range(TEST_NUM)]

print(TEST_TASKS)

['35 + 8 =', '93 + 69 =', '27 + 56 =', '46 + 57 =', '81 + 51 =', '31 + 32 =', '22 + 14 =', '29 + 2 =', '40 + 81 =', '31 + 13 =', '82 + 86 =', '11 + 69 =', '51 + 84 =', '65 + 71 =', '29 + 65 =', '7 + 65 =', '71 + 45 =', '67 + 15 =', '21 + 78 =', '97 + 92 =']


In [232]:
def score(tasks, answers):
    n = len(tasks)
    correct = 0

    for i in range(len(tasks)):
        task = tasks[i]
        pred_ans = answers[i]
        num1, operator, num2, _  = task.split()

        num1 = int(num1)
        num2 = int(num2)

        match operator:
            case "+":
                ans = num1 + num2
            case "-":
                ans = num1 - num2
            case "*":
                ans = num1 * num2
            case "/":
                ans = num1 / num2
            case _:
                raise 

        if (pred_ans == ans):
            correct += 1

    return f"{(correct / n) * 100:.01f} %"

In [233]:
def random_answer(task: str) -> str:
    num1, operator, num2, _  = task.split()

    num1 = int(num1)
    num2 = int(num2)

    match operator:
        case "+":
            min_val = min(num1, num2, num1 + num2)
            max_val = max(num1, num2, num1 + num2)
        case "-":
            min_val = min(num1, num2, num1 - num2)
            max_val = max(num1, num2, num1 - num2)
        case "*":
            min_val = min(num1, num2, num1 * num2)
            max_val = max(num1, num2, num1 * num2)
        case "/":
            min_val = min(num1, num2, num1 / num2)
            max_val = max(num1, num2, num1 / num2)
        case _:
            raise 

    ran_ans = random.randint(min_val, max_val)

    return ran_ans

In [234]:
def build_prompt(task: str) -> str:
    prompt = (
        "Podaj wynik każdego działania.\n"
        "Przykłady:\n"
    )
    for ex in EXAMPLE_FEWSHOTS:
        prompt += ex

    prompt += "\nTeraz rozwiąż to działanie:\n"
    prompt += f"{task}"

    return prompt


def generate_answer(task: str, max_new_tokens: int = 3) -> str:
    prompt = build_prompt(task)
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,          # wymuszamy greedy
            pad_token_id=tokenizer.eos_token_id
        )

    full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Zwrot tylko tego, co jest po '='
    return full_text[len(prompt):].strip()

In [235]:
random_answers = []

for task in TEST_TASKS:
    ans = random_answer(task)
    random_answers.append(int(ans))
    print(f"{task} {ans}")

print(score(TEST_TASKS, random_answers))

35 + 8 = 29
93 + 69 = 99
27 + 56 = 68
46 + 57 = 103
81 + 51 = 129
31 + 32 = 38
22 + 14 = 19
29 + 2 = 18
40 + 81 = 80
31 + 13 = 31
82 + 86 = 136
11 + 69 = 66
51 + 84 = 92
65 + 71 = 132
29 + 65 = 68
7 + 65 = 45
71 + 45 = 79
67 + 15 = 29
21 + 78 = 64
97 + 92 = 121
5.0 %


In [236]:
# --------------------------
# Test – 5 prostych zadań
# --------------------------

print(f"\n=== Test {TEST_NUM} przykładów ===\n")

answers = []

for task in TEST_TASKS:
    ans = generate_answer(task)
    answers.append(parse_number(ans))
    print(f"{task} {parse_number(ans)}")

print(score(TEST_TASKS, answers))


=== Test 20 przykładów ===

35 + 8 = 8
93 + 69 = 1
27 + 56 = 1
46 + 57 = 1
81 + 51 = 1
31 + 32 = 8
22 + 14 = 8
29 + 2 = 4
40 + 81 = 1
31 + 13 = 1
82 + 86 = 1
11 + 69 = 1
51 + 84 = 1
65 + 71 = 1
29 + 65 = 1
7 + 65 = 1
71 + 45 = 1
67 + 15 = 1
21 + 78 = 1
97 + 92 = 1
0.0 %


# Wnioski